In [105]:
import camelot as c
import os
import pandas as pd

files = os.listdir(r"C:\Users\there\Documents\Code\Numismatics\OCR\pdfs\simple")
pre = r"C:\Users\there\Documents\Code\Numismatics\OCR\pdfs\simple\\"

In [106]:
# for file in files:
#     pdf = pre + file
#     tables = c.read_pdf(pdf, "all")
#     for table in tables:
#         if table.parsing_report["accuracy"] > 0.90:
#             dir = f"./data/{file}"
#             if not os.path.exists(dir):
#                 os.mkdir(dir)
#             table.to_csv(dir + f"/{table.parsing_report['order']}.csv")

# Single-File

In [108]:
testPDF = pre + "Ariel and Berman - The Coins from Khirbat Burin.pdf"
tables = c.read_pdf(testPDF, "all", flavor="lattice", line_scale=40, line_tol=12)

### Remove Duplicate Headers

In [109]:
overall_df = tables[0].df
for table in tables[1:]:
    cur_df = table.df

    if cur_df[0][0] == overall_df[0][0]: # if non-initial table includes header again...
        cur_df = cur_df.drop(0) # ...remove the header

    overall_df = overall_df.merge(cur_df, how="outer")

In [110]:
cur_group = None
new_col = []

group_indices = []

for index, row in overall_df.iterrows():
    if sum(value != "" for value in row) == 1: # if only one cell filled --- indicates this is a grouping label/separator
        cur_group = next(value for value in row if value != "")
        group_indices.append(index)
    else:
        new_col.append(cur_group)

for index in group_indices:
    overall_df = overall_df.drop(index)

overall_df = overall_df.assign(group=new_col) # append new groups column

In [114]:
overall_df.to_csv("CUR.csv", index=False, header=False, encoding="utf-8") # need to add label for group depending on user input

In [112]:
# print(table.parsing_report)
# table = tables[0]
# c.plot(table, kind="contour") # line joint, text grid contour, textedge (Lat, Both, Stream)